In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext("local","blr")
spark = SparkSession.builder.appName("blr").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 21:29:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.7/dist-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
schema = StructType(
    [
        StructField("SqFt", IntegerType(), True),
        StructField("Rooms", IntegerType(), True),
        StructField("Baths", IntegerType(), True),
        StructField("Price", IntegerType(), True),
    ]
)
data = [
    (1500,3,2,320000), (2800,4,3,550000), (2200,3,2,450000), (3500,5,4,720000), 
    (1800,3,2,380000), (4000,5,3,850000), (2500,4,3,520000), (3200,4,3,680000), 
]
df = spark.createDataFrame(data, schema)
df.show(3)

+----+-----+-----+------+
|SqFt|Rooms|Baths| Price|
+----+-----+-----+------+
|1500|    3|    2|320000|
|2800|    4|    3|550000|
|2200|    3|    2|450000|
+----+-----+-----+------+
only showing top 3 rows



In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [4]:
feature_columns = ["SqFt","Rooms","Baths"]
assembler = VectorAssembler(inputCols = feature_columns, outputCol = "features")
df_assembled = assembler.transform(df)
df_assembled.show(3)

+----+-----+-----+------+----------------+
|SqFt|Rooms|Baths| Price|        features|
+----+-----+-----+------+----------------+
|1500|    3|    2|320000|[1500.0,3.0,2.0]|
|2800|    4|    3|550000|[2800.0,4.0,3.0]|
|2200|    3|    2|450000|[2200.0,3.0,2.0]|
+----+-----+-----+------+----------------+
only showing top 3 rows



In [ ]:
train_df, test_df = df_assembled.randomSplit([0.7,0.3])
lr = LogisticRegression(featuresCol="features", labelCol= "Price")
model = lr.fit(train_df)